In [1]:
# ---- load globals ----
source("R/globals.R", local = TRUE)

source("R/data_loading.R", local = TRUE)


source("R/module_rotate.R", local = TRUE)

library(plotly)       # ← 新增
library(base64enc)    # ← 新增（如果保留 data URI 路线）
source("R/module_select.R") 



Attaching package: ‘jsonlite’


The following object is masked from ‘package:shiny’:

    validate



Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout




In [ ]:
# # ---- Shiny UI ----

ui <- fluidPage(
  titlePanel("Visium HE overlay + gene expression (Lv0)"),
  sidebarLayout(
    sidebarPanel(
      # 上传控件
      fileInput("matrix_file",   "上传 matrix.mtx(.gz)"),
      fileInput("barcodes_file", "上传 barcodes.tsv(.gz)"),
      fileInput("features_file", "上传 features.tsv(.gz)"),
      fileInput("positions_file","上传 tissue_positions_list.csv"),
      fileInput("scalef_file",   "上传 scalefactors_json.json"),
      fileInput("image_file",    "上传 HE 图像 (png/jpg)"),

      # 基因选择：初始为空，加载后用 server 端 updateSelectInput() 填
      selectInput("gene", "选择基因：", choices = character(0), selected = NULL),


      checkboxInput("log1p", "log1p 变换", value = TRUE),
      sliderInput("ptsize", "点大小", min = 1, max = 6, value = 3, step = 0.5),
      sliderInput("alpha", "点透明度", min = 0.1, max = 1, value = 0.9, step = 0.1),
      helpText("请先上传 10x 矩阵三件套（matrix/barcodes/features）再选择基因。"),

      tags$hr(),
      rotateImageUI("rot")
    ),


    mainPanel(
      # 选择工具（套索/矩形）
      radioButtons(
        "sel_mode", "选择工具",
        choices = c("套索 (lasso)" = "lasso", "矩形 (box)" = "select"),
        inline = TRUE, selected = "lasso"
      ),

      # 交互主图：HE 叠底 + spots（可圈选）
      plotly::plotlyOutput("he_plotly", height = "750px", width = "100%"),

      # 操作区：清空/下载/计数
      fluidRow(
        column(4, actionButton("clear_sel", "清空选择")),
        column(4, downloadButton("dl_sel", "下载所选 CSV")),
        column(4, verbatimTextOutput("sel_count"))
      ),

      hr(),
      verbatimTextOutput("dbg")
    )
    )
  )


In [ ]:
# ---- 读图：确保返回 raster ----
load_image <- function(path) {
  if (grepl("\\.png$", path, ignore.case = TRUE)) {
    arr <- png::readPNG(path)     # H x W x C, 0..1
  } else if (grepl("\\.jpe?g$", path, ignore.case = TRUE)) {
    arr <- jpeg::readJPEG(path)
  } else stop("Only PNG/JPG supported.")

  # 灰度图转 3 通道
  if (length(dim(arr)) == 2L) {
    arr <- array(rep(arr, each = 3), dim = c(dim(arr), 3))
  }
  img_h <- dim(arr)[1]; img_w <- dim(arr)[2]
  img_rs <- as.raster(arr)        # 关键：转 raster
  list(img = img_rs, img_w = img_w, img_h = img_h)
}

server <- function(input, output, session) {
  rv <- reactiveValues(
    counts=NULL, gene_names=NULL, barcodes=NULL,
    pos=NULL, hires_scale=NULL, img=NULL, img_w=NULL, img_h=NULL, img_path=NULL
  )

  # ---- 加载 counts/features/barcodes ----
  observeEvent(list(input$matrix_file, input$barcodes_file, input$features_file), {
    req(input$matrix_file, input$barcodes_file, input$features_file)
    paths <- list(input$matrix_file$datapath, input$barcodes_file$datapath, input$features_file$datapath)
    message("Loading counts ...")
    res <- load_counts(paths[[1]], paths[[2]], paths[[3]])
    rv$counts     <- res$counts
    rv$gene_names <- res$gene_names
    rv$barcodes   <- res$barcodes
    updateSelectInput(session, "gene", choices = head(rv$gene_names, MAX_GENES),
                      selected = head(rv$gene_names, 1))
  })

  # ---- 加载 positions ----
  observeEvent(input$positions_file, {
    req(input$positions_file)
    message("Loading positions ...")
    pos <- load_positions(input$positions_file$datapath)
    if (!is.null(rv$counts)) {
      pos <- subset(pos, in_tissue == 1 & barcode %in% colnames(rv$counts))
    } else {
      pos <- subset(pos, in_tissue == 1)
    }
    rv$pos <- pos
  })

  # ---- 加载 scalefactor ----
  observeEvent(input$scalef_file, {
    req(input$scalef_file)
    message("Loading scalefactor ...")
    rv$hires_scale <- load_scalef(input$scalef_file$datapath)
    if (!is.null(rv$pos)) rv$pos <- to_hires_coords(rv$pos, rv$hires_scale)
  })

  # ---- 加载图像 ----
  observeEvent(input$image_file, {
    req(input$image_file)
    message("Loading image ...")
    im <- load_image(input$image_file$datapath)
    rv$img   <- im$img      # raster
    rv$img_w <- im$img_w
    rv$img_h <- im$img_h
    rv$img_path <- input$image_file$datapath
  })

  # ---- pos 更新后补映射 ----
  observeEvent(rv$pos, {
    if (!is.null(rv$pos) && !is.null(rv$hires_scale) && is.null(rv$pos$x_hires)) {
      rv$pos <- to_hires_coords(rv$pos, rv$hires_scale)
    }
  })

  # ---- 表达向量 ----
  expr_vec <- reactive({
    req(input$gene, rv$counts, rv$pos)
    v <- if (input$gene %in% rownames(rv$counts)) {
      as.numeric(rv$counts[input$gene, rv$pos$barcode])
    } else rep(0, nrow(rv$pos))
    if (isTRUE(input$log1p)) v <- log1p(v)
    v
  })

  rot <- rotateImageServer("rot", image_path_reactive = reactive(rv$img_path))

  current_img_obj <- reactive({
    # 优先用旋转后的；否则用原图
    if (!is.null(rot$get())) {
      rot$get()
    } else if (!is.null(rv$img)) {
      list(img = rv$img, img_w = rv$img_w, img_h = rv$img_h)
    } else {
      NULL
    }
  })

sel <- selectSpotsServer(
    "sel",
    pos_reactive  = reactive(rv$pos),      # 需要包含 barcode/x_hires/y_hires
    expr_reactive = expr_vec,              # 你已有的表达向量 reactive
    img_reactive  = current_img_obj        # 背景图（旋转后/原图皆可）
  )


  raster_to_data_uri <- function(rs, width, height) {
    im <- magick::image_read(rs)
    im <- magick::image_resize(im, paste0(width, "x", height, "!"))  # 强制到 W x H
    tf <- tempfile(fileext = ".png")
    magick::image_write(im, path = tf, format = "png")
    raw <- readBin(tf, what = "raw", n = file.info(tf)$size)
    uri <- paste0("data:image/png;base64,", base64enc::base64encode(raw))
    uri
  }

  # 当前背景（优先旋转后）
  current_img_obj <- reactive({
    ro <- tryCatch(rot$get(), error = function(e) NULL)
    if (!is.null(ro)) ro else if (!is.null(rv$img)) list(img=rv$img, img_w=rv$img_w, img_h=rv$img_h) else NULL
  })

  # 主图：HE 底图 + spot；在同图内圈选
  output$he_plotly <- plotly::renderPlotly({
    req(rv$pos, current_img_obj())
    img_obj <- current_img_obj()
    W <- img_obj$img_w; H <- img_obj$img_h

    # spot 数据
    v  <- expr_vec()
    df <- data.frame(
      idx = seq_len(nrow(rv$pos)),
      barcode = rv$pos$barcode,
      x = rv$pos$x_hires,
      y = rv$pos$y_hires,
      expr = v,
      stringsAsFactors = FALSE
    )

    # 背景 data URI
    bg_uri <- raster_to_data_uri(img_obj$img, W, H)

    plotly::plot_ly(
      data = df,
      x = ~x, y = ~y,
      type = "scattergl", mode = "markers",
      marker = list(size = input$ptsize, opacity = input$alpha,
                    color = ~expr, colorscale = "Viridis",
                    colorbar = list(title = input$gene %||% "expression")),
      text = ~paste0("barcode: ", barcode,
                    "<br>x: ", round(x,1),
                    "<br>y: ", round(y,1),
                    "<br>expr: ", signif(expr,4)),
      hoverinfo = "text",
      source = "main",                   # ★ 绑定一个 source
      customdata = ~idx                  # ★ 每个点带上自己的行号
    ) %>%
      plotly::layout(
        xaxis = list(range = c(0, W), zeroline = FALSE, showgrid = FALSE),
        yaxis = list(range = c(H, 0), zeroline = FALSE, showgrid = FALSE,
                    scaleanchor = "x", scaleratio = 1),
        images = list(
          list(source = bg_uri, xref="x", yref="y",
              x = 0, y = 0, sizex = W, sizey = H,
              sizing = "stretch", layer = "below")
        ),
        dragmode = if (isTruthy(input$sel_mode) && input$sel_mode=="select") "select" else "lasso",
        margin = list(l=0, r=0, t=0, b=0),
        showlegend = FALSE
      )
  })

  # 监听选择事件（同一个 source）
  selected_idx <- reactiveVal(integer(0))
  observeEvent(plotly::event_data("plotly_selected", source = "main"), {
    ed <- plotly::event_data("plotly_selected", source = "main")
    if (is.null(ed) || nrow(ed)==0) {
      selected_idx(integer(0))
    } else {
      idx <- ed$customdata
      if (is.null(idx)) idx <- ed$pointNumber + 1L
      selected_idx(sort(unique(as.integer(idx))))
    }
  })

  # 根据单选切换 dragmode（可选）
  observeEvent(input$sel_mode, {
    plotly::plotlyProxy("he_plotly", session) %>%
      plotly::plotlyProxyInvoke("relayout",
        list(dragmode = if (input$sel_mode=="select") "select" else "lasso"))
  })

  # 清空选择
  observeEvent(input$clear_sel, {
    selected_idx(integer(0))
    plotly::plotlyProxy("he_plotly", session) %>%
      plotly::plotlyProxyInvoke("restyle", list(selectedpoints = list(NULL)))
  })

  # 已选数量 + 下载
  output$sel_count <- renderText({
    paste0("已选 spots: ", length(selected_idx()))
  })

  output$dl_sel <- downloadHandler(
    filename = function() paste0("selected_spots_", format(Sys.time(), "%Y%m%d_%H%M%S"), ".csv"),
    content  = function(file) {
      req(rv$pos)
      idx <- selected_idx()
      if (length(idx)==0) {
        write.csv(rv$pos[0, c("barcode","x_hires","y_hires")], file, row.names = FALSE)
      } else {
        v  <- expr_vec()
        out <- data.frame(
          barcode = rv$pos$barcode[idx],
          x_hires = rv$pos$x_hires[idx],
          y_hires = rv$pos$y_hires[idx],
          expr    = v[idx],
          stringsAsFactors = FALSE
        )
        utils::write.csv(out, file, row.names = FALSE)
      }
    }
  )



  # ---- 调试信息（可留可去） ----
  output$dbg <- renderPrint({
    list(
      have_counts = !is.null(rv$counts),
      have_pos    = !is.null(rv$pos),
      have_scale  = !is.null(rv$hires_scale),
      have_img    = !is.null(rv$img),
      img_class   = if (!is.null(rv$img)) class(rv$img)
    )
  })
}


In [5]:
shinyApp(ui, server)


Listening on http://127.0.0.1:7144

Warning message:
“The 'plotly_selected' event tied a source ID of '' is not registered. In order to obtain this event data, please add `event_register(p, 'plotly_selected')` to the plot (`p`) that you wish to obtain event data from.”
Warning message:
“The 'plotly_selected' event tied a source ID of 'main' is not registered. In order to obtain this event data, please add `event_register(p, 'plotly_selected')` to the plot (`p`) that you wish to obtain event data from.”
Warning message:
“The 'plotly_selected' event tied a source ID of 'main' is not registered. In order to obtain this event data, please add `event_register(p, 'plotly_selected')` to the plot (`p`) that you wish to obtain event data from.”
